In [169]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import pandas as pd
from scipy.spatial import distance

In [170]:
#glove = pd.read_csv("data/glove.6B.50d.txt", delimiter=' ', header=None, quoting=3)
ladec = pd.read_csv("data/ladec.csv")

In [63]:
embeddings_dict = {}

with open("data/glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [95]:
def getv(name):
    word = glove[glove[0]==name]
    return np.array( word.values[0][1:] )

def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))

In [64]:
glove.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,the,0.418000,0.249680,-0.41242,0.12170,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,...,-0.298710,-0.157490,-0.347580,-0.045637,-0.44251,0.187850,0.002785,-0.184110,-0.115140,-0.78581
1,",",0.013441,0.236820,-0.16899,0.40951,0.63812,0.477090,-0.42852,-0.55641,-0.36400,...,-0.080262,0.630030,0.321110,-0.467650,0.22786,0.360340,-0.378180,-0.566570,0.044691,0.30392
2,.,0.151640,0.301770,-0.16763,0.17684,0.31719,0.339730,-0.43478,-0.31086,-0.44999,...,-0.000064,0.068987,0.087939,-0.102850,-0.13931,0.223140,-0.080803,-0.356520,0.016413,0.10216
3,of,0.708530,0.570880,-0.47160,0.18048,0.54449,0.726030,0.18157,-0.52393,0.10381,...,-0.347270,0.284830,0.075693,-0.062178,-0.38988,0.229020,-0.216170,-0.225620,-0.093918,-0.80375
4,to,0.680470,-0.039263,0.30186,-0.17792,0.42962,0.032246,-0.41376,0.13228,-0.29847,...,-0.094375,0.018324,0.210480,-0.030880,-0.19722,0.082279,-0.094340,-0.073297,-0.064699,-0.26044


In [179]:
c1 = [ ladec['c1'][x] for x in range( len(ladec['c1']) ) if ladec['correctParse'][x]=='yes' ]

c2 = [ ladec['c2'][x] for x in range( len(ladec['c2']) ) if ladec['correctParse'][x]=='yes' ]

cmp = [ ladec['stim'][x] for x in range( len(ladec['stim']) ) if ladec['correctParse'][x]=='yes' ]

#c2 = ladec['c2'].to_numpy(dtype = 'str')
#cmp = ladec['stim'].to_numpy(dtype = 'str')

In [181]:
len(c1), len(c2), len(cmp)

(8372, 8372, 8372)

In [35]:
#Parallel to cOne, cTwo, and cmp; at each index the Vec array contains the word embedding data from glove

gad = cOne[0]
gad

'gad'

In [69]:
gadv = getv( gad )
gadv

array([-0.28186, 0.76122, 0.31124, -0.056947, -0.16125, -0.84541, 0.21271,
       -1.1618, 0.66181, 1.2302, 0.78864, 0.83112, -0.70775, 0.22583,
       -0.7578, 1.221, 0.69907, -0.81823, 0.19546, 1.8109, -0.71717,
       0.87639, 0.68515, -0.10325, -0.12523, 0.1229, 0.056091, -0.18191,
       0.036424, 0.43651, -0.5549, 0.35513, 0.26945, 0.58567, 0.6719,
       -0.52472, 0.59862, -0.47292, 0.33516, 1.8675, 0.079179, 1.2164,
       -0.23826, 0.0051595, 0.44026, -0.63698, -0.24248, -0.38488,
       0.11777, 0.2475], dtype=object)

In [57]:
#These should be something that is easy to write/read as csv!
#Does pandas have an appropriate data structure?

c1_vec = {}
c2_vec = {}
cmp_vec = {}

In [67]:
for i in range(0, len(c1) ):
    #print(c1[i])
    
    if c1[i] not in glove[0:][0]:
        c1_dict[ c1[i] ] = []
    else:
        c1_dict[ c1[i] ] = glove[ c1[i] ]

In [72]:
pd?

In [74]:
c1?

In [182]:
lexicon = pd.concat( [ pd.Series(c1), pd.Series(c2), pd.Series(cmp) ] , axis=0)

In [183]:
lexicon.shape

(25116,)

In [184]:
lexicon.unique().shape

(11786,)

In [185]:
words = set(embeddings_dict.keys()).intersection(lexicon.unique())

In [186]:
len(words)

9302

In [187]:
c1_vec = { x: embeddings_dict.get(x) for x in pd.Series(c1).unique() }
c2_vec = { x: embeddings_dict.get(x) for x in pd.Series(c2).unique() }
cmp_vec = { x: embeddings_dict.get(x) for x in pd.Series(cmp).unique() }

#When there is no embedding availible dict[word] = None
#Guard for this when gathering data

In [188]:
c2_vec

{'about': array([ 0.89466 ,  0.36604 ,  0.37588 , -0.41818 ,  0.58462 ,  0.18594 ,
        -0.41907 , -0.46621 , -0.54903 ,  0.02477 , -0.90816 , -0.48271 ,
        -0.050742, -0.74039 ,  1.4377  , -0.01974 , -0.2384  ,  0.43154 ,
        -0.6612  , -0.41275 ,  0.25475 ,  0.93498 ,  0.81404 , -0.17296 ,
         0.61296 , -1.8475  , -0.27616 ,  0.27701 ,  0.42347 , -0.11599 ,
         3.6243  ,  0.12306 , -0.023526, -0.24843 , -0.22376 , -0.53941 ,
        -0.62444 , -0.27711 ,  0.49406 ,  0.020234, -0.2346  ,  0.44512 ,
         0.53397 ,  0.66654 , -0.093662, -0.035203, -0.064194,  0.55998 ,
        -0.66593 ,  0.12177 ], dtype=float32),
 'abouts': array([ 0.55139 ,  0.67295 ,  0.35237 , -0.95209 , -1.0637  , -0.57586 ,
         0.519   , -0.38231 ,  0.541   ,  0.30251 , -1.3331  , -1.0013  ,
         0.17228 ,  0.25971 ,  0.071468,  0.088483,  0.85305 , -0.15511 ,
         0.79294 ,  0.44542 , -0.6149  ,  0.35353 ,  0.68477 ,  0.32059 ,
         0.72273 ,  1.0649  , -0.40679 ,  0.24

In [96]:
c1?

In [204]:
c1_ori = np.unique(c1, return_counts=True )

In [205]:
c1_sorted = np.sort( c1_ori[1] )
c1_sorted

array([  1,   1,   1, ...,  84, 100, 129])

In [206]:
list( c1_ori[1] ).index(129)

1115

In [207]:
common_word_1 = c1_ori[0][1115]
common_word_1

'over'

In [199]:
c2_ori = np.unique(c2, return_counts=True)

In [200]:
c2_sorted = np.sort( c2_ori[1] )
c2_sorted

array([  1,   1,   1, ..., 133, 147, 228])

In [201]:
list( c2_ori[1] ).index(228)

1148

In [202]:
common_word_2 = c2_ori[0][1148]
common_word_2

'man'

In [208]:
np.where( cmp=='overman' )

#The two most common constituents do not go together...
#Tragic...

(array([], dtype=int64),)

In [203]:
np.max( c2_ori[1] )

228

In [128]:
np.where( c2_ori[1]==229 )

(array([1235]),)

In [130]:
c2_ori[0][1235]

'man'

In [209]:
np.where( c2=='man')

#All compounds with form: fooman

(array([], dtype=int64),)

In [132]:
cmp[ Out[131] ]

array(['tallyman', 'woodman', 'horseman', 'townsman', 'ploughman',
       'groundsman', 'aircraftman', 'sidesman', 'bagman', 'batsman',
       'groomsman', 'salesman', 'jazzman', 'cowman', 'brakeman',
       'muscleman', 'hardwareman', 'seedman', 'pitman', 'coalman',
       'poultryman', 'cattleman', 'lineman', 'newsman', 'selectman',
       'dalesman', 'tollman', 'craftsman', 'raftsman', 'gagman',
       'bogeyman', 'policeman', 'lawman', 'guardsman', 'bandsman',
       'strongman', 'raftman', 'coachman', 'signalman', 'countryman',
       'linesman', 'roadman', 'houseman', 'backwoodsman', 'militiaman',
       'ledgeman', 'iceman', 'churchman', 'serviceman', 'bushman',
       'cavalryman', 'snowman', 'milkman', 'handyman', 'midshipman',
       'lensman', 'pointsman', 'washerman', 'underclassman', 'clansman',
       'bondsman', 'bondman', 'gentleman', 'ferryman', 'landman',
       'lockman', 'rifleman', 'headsman', 'doorman', 'plowman',
       'yachtsman', 'propman', 'workman', 'chairma

In [210]:
foomen = Out[132]

In [211]:
len( words.intersection( foomen ) )

175

In [219]:
embedded_foomen = words.intersection( foomen )

In [220]:
embedded_foomen

{'aircraftman',
 'airman',
 'alderman',
 'anchorman',
 'artilleryman',
 'assemblyman',
 'backwoodsman',
 'bagman',
 'bandsman',
 'barman',
 'batman',
 'batsman',
 'bellman',
 'boatman',
 'bogeyman',
 'bondsman',
 'bookman',
 'bowman',
 'brakeman',
 'bushman',
 'businessman',
 'busman',
 'cameraman',
 'cattleman',
 'cavalryman',
 'caveman',
 'chairman',
 'chapman',
 'chessman',
 'churchman',
 'clansman',
 'clergyman',
 'coachman',
 'committeeman',
 'congressman',
 'councilman',
 'countryman',
 'cowman',
 'craftsman',
 'crewman',
 'dairyman',
 'deliveryman',
 'doorman',
 'draftsman',
 'draughtsman',
 'dustman',
 'earthman',
 'ferryman',
 'fieldsman',
 'fireman',
 'fisherman',
 'footman',
 'freedman',
 'freeman',
 'freshman',
 'frogman',
 'frontiersman',
 'garbageman',
 'gasman',
 'gentleman',
 'groomsman',
 'groundsman',
 'guardsman',
 'gunman',
 'handyman',
 'hangman',
 'harvestman',
 'headman',
 'headsman',
 'helmsman',
 'herdsman',
 'highwayman',
 'hitman',
 'horseman',
 'houseman',
 

In [221]:
np.where( cmp == embedded_foomen )

(array([], dtype=int64),)

In [222]:
[ x for x in np.array( [np.where(cmp==x) for x in embedded_foomen ], dtype='object' ).flat ]

[]

In [223]:
foomen_indexes = [ x for x in range( len(cmp) ) if cmp[x] in embedded_foomen]
len(foomen_indexes)

175

In [161]:
c1[foomen_indexes]

array(['rep', 'ware', 'pol', 'wood', 'horse', 'towns', 'plough',
       'grounds', 'aircraft', 'bag', 'bats', 'grooms', 'sales', 'jazz',
       'cow', 'brake', 'muscle', 'seed', 'pit', 'cattle', 'line', 'news',
       'select', 'toll', 'crafts', 'bogey', 'police', 'law', 'guards',
       'bands', 'strong', 'coach', 'signal', 'country', 'lines', 'road',
       'house', 'backwoods', 'militia', 'ice', 'church', 'service',
       'bush', 'cavalry', 'snow', 'milk', 'handy', 'midship', 'lens',
       'underclass', 'clans', 'bonds', 'gentle', 'ferry', 'land', 'lock',
       'rifle', 'heads', 'door', 'plow', 'yachts', 'work', 'chair',
       'letter', 'sports', 'freed', 'infantry', 'herds', 'timber', 'dust',
       'wing', 'marks', 'lobster', 'committee', 'lands', 'spokes',
       'warehouse', 'bat', 'watch', 'newspaper', 'free', 'drafts',
       'draughts', 'iron', 'harvest', 'cave', 'train', 'patrol', 'pig',
       'fire', 'fisher', 'money', 'steel', 'water', 'clergy', 'council',
       'ald

In [160]:
np.where( cmp=='repman' )

(array([], dtype=int64),)

In [215]:
foomen_indexes

[68,
 3658,
 3728,
 4527,
 4528,
 4529,
 4530,
 4531,
 4532,
 4534,
 4535,
 4536,
 4537,
 4538,
 4539,
 4540,
 4541,
 4543,
 4544,
 4547,
 4548,
 4549,
 4550,
 4552,
 4553,
 4556,
 4557,
 4558,
 4559,
 4560,
 4561,
 4563,
 4564,
 4565,
 4566,
 4567,
 4568,
 4569,
 4570,
 4572,
 4573,
 4574,
 4575,
 4576,
 4577,
 4578,
 4579,
 4580,
 4581,
 4584,
 4585,
 4586,
 4588,
 4589,
 4590,
 4591,
 4592,
 4593,
 4594,
 4595,
 4596,
 4598,
 4599,
 4600,
 4601,
 4602,
 4604,
 4605,
 4606,
 4608,
 4609,
 4610,
 4611,
 4612,
 4613,
 4614,
 4615,
 4616,
 4619,
 4620,
 4622,
 4623,
 4625,
 4626,
 4627,
 4628,
 4629,
 4630,
 4631,
 4632,
 4633,
 4636,
 4638,
 4639,
 4640,
 4641,
 4643,
 4644,
 4645,
 4646,
 4647,
 4648,
 4651,
 4652,
 4653,
 4654,
 4655,
 4656,
 4657,
 4658,
 4659,
 4660,
 4663,
 4664,
 4665,
 4666,
 4667,
 4671,
 4673,
 4674,
 4675,
 4676,
 4678,
 4679,
 4680,
 4681,
 4682,
 4683,
 4684,
 4685,
 4687,
 4688,
 4691,
 4692,
 4693,
 4695,
 4696,
 4697,
 4698,
 4699,
 4700,
 4702,
 4703,
 

In [216]:
cmp[68]

'potash'

In [217]:
c1[68]

'pot'

In [218]:
c2[68]

'ash'